In [ ]:
import os
import requests
import json
import re
from urllib.parse import unquote

API_URL = "https://en.wikivoyage.org/w/api.php"

def fetch_page_html(page_title):
    """
    Fetch HTML content of a WikiVoyage page to extract links
    """
    params = {
        'action': 'query',
        'format': 'json',
        'titles': page_title,
        'prop': 'extracts',
        'explaintext': False,  # Get HTML to extract links
        'exsectionformat': 'wiki'
    }
    
    headers = {
        'User-Agent': 'ShivYatra Travel App (Educational Purpose)'
    }
    
    try:
        response = requests.get(API_URL, params=params, headers=headers)
        response.raise_for_status()
        
        data = response.json()
        pages = data['query']['pages']
        page_id = list(pages.keys())[0]
        page_data = pages[page_id]
        
        if 'extract' in page_data:
            return page_data['extract']
        return None
            
    except Exception as e:
        print(f"Error fetching {page_title}: {e}")
        return None

In [ ]:
def extract_regions_links(page_title):
    """
    Extract links from the Regions section of a WikiVoyage page
    """
    # First get the plain text to check if regions section exists
    page_data = fetch_page_extract(page_title)
    if not page_data:
        return []
    
    sections = parse_sections_from_extract(page_data['extract'])
    
    # Check if Regions section exists
    if 'Regions' not in sections:
        return []
    
    # Get HTML content to extract actual links
    html_content = fetch_page_html(page_title)
    if not html_content:
        return []
    
    soup = BeautifulSoup(html_content, 'html.parser')
    
    # Find all internal WikiVoyage links
    links = []
    for link in soup.find_all('a', href=True):
        href = link['href']
        # Look for WikiVoyage internal links
        if href.startswith('/wiki/') and ':' not in href:
            # Clean up the link
            page_name = href.replace('/wiki/', '').replace('_', ' ')
            page_name = unquote(page_name)  # Decode URL encoding
            links.append(page_name)
    
    # Filter to get only links that appear in the regions section content
    regions_content = sections['Regions'].lower()
    regions_links = []
    
    for link in links:
        if link.lower() in regions_content or link.replace(' ', '_').lower() in regions_content:
            regions_links.append(link)
    
    return list(set(regions_links))  # Remove duplicates

In [ ]:
def find_final_destinations(page_title, visited=None, max_depth=5):
    """
    Recursively find all final destination pages (pages without Regions section)
    """
    if visited is None:
        visited = set()
    
    if page_title in visited or max_depth <= 0:
        return []
    
    visited.add(page_title)
    print(f"Exploring: {page_title}")
    
    # Get page data
    page_data = fetch_page_extract(page_title)
    if not page_data:
        return []
    
    sections = parse_sections_from_extract(page_data['extract'])
    
    # If no Regions section, this is a final destination
    if 'Regions' not in sections:
        print(f"  → Final destination found: {page_title}")
        return [page_title]
    
    # If has Regions section, explore the linked pages
    final_destinations = []
    regions_links = extract_regions_links(page_title)
    
    print(f"  → Found {len(regions_links)} regions to explore")
    
    for link in regions_links:
        if link not in visited:
            final_destinations.extend(
                find_final_destinations(link, visited, max_depth - 1)
            )
    
    return final_destinations

In [ ]:
# Explore all states in him_north_list and find final destinations
state_destinations = {}

print("🔍 Starting exploration of Himalayan North states...")
print("=" * 60)

for state in him_north_list:
    print(f"\n🏔️  EXPLORING STATE: {state}")
    print("-" * 50)
    
    destinations = find_final_destinations(state)
    state_destinations[state] = destinations
    
    print(f"\n✅ Found {len(destinations)} final destinations for {state}")
    print(f"Destinations: {destinations[:5]}{'...' if len(destinations) > 5 else ''}")

print("\n🎉 Exploration complete!")
print(f"Total states explored: {len(state_destinations)}")
for state, destinations in state_destinations.items():
    print(f"  {state}: {len(destinations)} destinations")

In [ ]:
# Create separate lists for each state (as requested)

# Himachal Pradesh destinations
himachal_pradesh_destinations = state_destinations.get("Himachal_Pradesh", [])
print(f"Himachal Pradesh destinations ({len(himachal_pradesh_destinations)}):")
print(himachal_pradesh_destinations)

print("\n" + "="*60 + "\n")

# Jammu and Kashmir destinations  
jammu_kashmir_destinations = state_destinations.get("Jammu_and_Kashmir", [])
print(f"Jammu and Kashmir destinations ({len(jammu_kashmir_destinations)}):")
print(jammu_kashmir_destinations)

print("\n" + "="*60 + "\n")

# Ladakh destinations
ladakh_destinations = state_destinations.get("Ladakh", [])
print(f"Ladakh destinations ({len(ladakh_destinations)}):")
print(ladakh_destinations)

print("\n" + "="*60 + "\n")

# Uttarakhand destinations
uttarakhand_destinations = state_destinations.get("Uttarakhand", [])
print(f"Uttarakhand destinations ({len(uttarakhand_destinations)}):")
print(uttarakhand_destinations)